In [1]:
from load_librairies import *
import xscale.signal.fitting as xsf
import seaborn as sns
from sklearn import linear_model

def is_jja(month):
    return (month >= 6) & (month <= 8)

def is_amj(month):
    return (month >= 4) & (month <= 6)

def is_mjj(month):
    return (month >= 5) & (month <= 7)

now = datetime.now()
date_time = now.strftime("%d/%m/%Y")
def nandetrend(y):
    ''' Remove the linear trend from the data '''
    x = np.arange(0,y.shape[0],1)
    
    not_nan_ind = ~np.isnan(y)
    m, b, r_val, p_val, std_err = stats.linregress(x[not_nan_ind],np.array(y)[not_nan_ind])
    
    
    y_detrended= np.array(y) - m*x
    
    return y_detrended



def nandetrend2(y):
    ''' Remove the linear trend from the data '''
    x = np.arange(0,y.shape[0],1)
    
    not_nan_ind = ~np.isnan(y)
    m, b, r_val, p_val, std_err = stats.linregress(x[not_nan_ind],np.array(y)[not_nan_ind])
    
    
    y_detrended_tmp= np.array(y) - m*x
    
    y_detrended  = xr.Dataset({'detrended': (['time'],y_detrended_tmp)}
                       ,coords={'time':y.time[:]})
    return y_detrended

######################
#      TO CHANGE     # 
######################
dir_hist_z23 = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/z20/atl3/'
path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/new_version/reviews/'
path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

In [2]:
model = ['ACCESS1-0', 'ACCESS1-3','CMCC-CESM',
         'CMCC-CMS','CMCC-CM','CNRM-CM5','CSIRO-Mk3-6-0',
        'GFDL-CM3', 'GFDL-ESM2M','GISS-E2-H-CC','GISS-E2-H',
        'GISS-E2-R-CC','GISS-E2-R','HadGEM2-CC','IPSL-CM5A-LR',
        'IPSL-CM5A-MR','IPSL-CM5B-LR','MIROC-ESM-CHEM','MIROC-ESM',
        'MIROC5','MPI-ESM-LR','MPI-ESM-MR','MRI-CGCM3','NorESM1-M']

## SST variability changes in JJA

In [3]:

## SSTa ##


ssta_atl3_hist_cmip5 = xr.open_dataset(path_data+'ssta_hist_monthly_CMIP5.nc')
ssta_atl3_ssp5_cmip5 = xr.open_dataset(path_data+'ssta_ssp5_monthly_CMIP5.nc')

ssta_atl3_hist_cmip6 = xr.open_dataset(path_data+'ssta_hist_monthly_CMIP6.nc')
ssta_atl3_ssp5_cmip6 = xr.open_dataset(path_data+'ssta_ssp585_monthly_CMIP6.nc')

In [4]:
## std SSTa in JJA ##

std_ssta_hist_CMIP5_jja = ssta_atl3_hist_cmip5.ssta.sel(time=is_jja(
    ssta_atl3_hist_cmip5['time.month'])).std(dim='time')

std_ssta_ssp5_CMIP5_jja = ssta_atl3_ssp5_cmip5.ssta.sel(time=is_jja(
    ssta_atl3_ssp5_cmip5['time.month'])).std(dim='time')

std_ssta_hist_CMIP6_jja = ssta_atl3_hist_cmip6.ssta.sel(time=is_jja(
    ssta_atl3_hist_cmip6['time.month'])).std(dim='time')

std_ssta_ssp5_CMIP6_jja = ssta_atl3_ssp5_cmip6.ssta.sel(time=is_jja(
    ssta_atl3_ssp5_cmip6['time.month'])).std(dim='time')

std_ssta_changes_cmip6 = std_ssta_ssp5_CMIP6_jja - std_ssta_hist_CMIP6_jja
std_ssta_changes_cmip5 = std_ssta_ssp5_CMIP5_jja - std_ssta_hist_CMIP5_jja



##  Thermocline depth bias SST in JJA in the ATL3

In [5]:


dir_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Data/'
data     = dir_data + 'ORA-S4/'


file_sst = 'global_SST_ORA_S4_1958_2017_montlhy.nc'
nc = xr.open_mfdataset(data+file_sst)
lon_oras = np.arange(-60,20,1)
lat_oras = np.arange(-30,30,1)
time_oras = nc.TIME
#sst_oras = nc.THETAO



dir_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Data/'
data = dir_data + 'ORA-S4/'
file = 'z20_depth_ORAS4_jan1958_dec_2017.nc'
#nc   = Dataset(data+file)
nc        = xr.open_mfdataset(data+file)
z20_tmp = nc.Z20

#print(nc.variables.keys())
## z23 data from jan 1958 to dec 2017.
#z20_tmp = nc.variables['Z20'][:696,:,:] # taking the data from  jan 1958 to dec 2015 because of the wind data
#nc.close()
z20tmp_2=np.ones((np.shape(z20_tmp)))*np.nan
#


for i in range(z20tmp_2.shape[1]):
    for j in range(z20tmp_2.shape[2]):
        if np.isfinite(z20_tmp[:,i,j]).all()==True:
            z20tmp_2[:,i,j] = nandetrend(z20_tmp[:,i,j])

X,Y = np.meshgrid(lon_oras,lat_oras) 
#
#
z20  = xr.Dataset({'z20': (['time','y','x'],z20tmp_2)}
                   ,coords={'time':np.array(time_oras),'lat':(['y','x'],np.array(Y)),'lon':(['y','x'],np.array(X))})




In [12]:


period_1_str = 1982
period_1_end = 2017

z20_atl3 = Atools.data_sub(z20,-20,0,-3,3).mean(dim='x').mean(dim='y')
z20_atl3 = z20_atl3.sel(time=slice(datetime(period_1_str, 1, 1),
                                               datetime(period_1_end, 12, 31)))
z20_atl3_mean_ORAS4 = z20_atl3.z20.mean(dim='time')
z20_atl3_jja_ORAS4 = z20_atl3.z20.sel(time=is_jja(z20_atl3['time.month'])).mean(dim='time')


z20_atl4 = Atools.data_sub(z20,-40,-20,-3,3).mean(dim='x').mean(dim='y')
z20_atl4 = z20_atl4.sel(time=slice(datetime(period_1_str, 1, 1),
                                               datetime(period_1_end, 12, 31)))
z20_atl4_mean_ORAS4 = z20_atl4.z20.mean(dim='time')
z20_atl4_jja_ORAS4 = z20_atl4.z20.sel(time=is_jja(z20_atl4['time.month'])).mean(dim='time')

Z20_slope_ORAS4 = z20_atl3_jja_ORAS4 - z20_atl4_jja_ORAS4

In [16]:

z20_bias_jja = np.ones((len(model)))
z20_bias = np.ones((len(model)))


period_str_1 = 1950
period_end_1 = 1999

for i in range(0,len(model)):
    print(model[i])
    z20 = xr.open_dataset(dir_hist_z23+'z20_Omon_'+model[i]+'_historical_r1i1p1_187001-200012_1deg_atl3.nc')

    try:

        z20 = z20.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        z20['time'] = z20.indexes['time'].to_datetimeindex()
        z20 = z20.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
        
    z20 = z20.z20  
    z20 = z20.mean(dim='lon').mean(dim='lat')
    z20_dtd = nandetrend2(z20)
    z20_dtd_mean = z20_dtd.detrended.mean(dim='time')
    z20_dtd_jja_mean = z20_dtd.detrended.sel(time=is_jja(z20_dtd.detrended['time.month'])).mean(dim='time')
    
    z20_bias_jja[i] = z20_dtd_jja_mean - z20_atl3_jja_ORAS4
    z20_bias[i] = z20_dtd_mean- z20_atl3_mean_ORAS4

ACCESS1-0
ACCESS1-3
CMCC-CESM
CMCC-CMS
CMCC-CM
CNRM-CM5
CSIRO-Mk3-6-0
GFDL-CM3
GFDL-ESM2M


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  app.launch_new_instance()
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  app.launch_new_instance()
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the

GISS-E2-H-CC
GISS-E2-H
GISS-E2-R-CC
GISS-E2-R
HadGEM2-CC


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  app.launch_new_instance()
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  app.launch_new_instance()
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from th

IPSL-CM5A-LR
IPSL-CM5A-MR
IPSL-CM5B-LR
MIROC-ESM-CHEM
MIROC-ESM
MIROC5
MPI-ESM-LR
MPI-ESM-MR
MRI-CGCM3
NorESM1-M


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  app.launch_new_instance()
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  app.launch_new_instance()


In [17]:
bias_jja_cmip5  = xr.Dataset({'bias': (['product'],z20_bias_jja),
                       }
                       ,coords={'product':model},attrs={'standard_name': 'JJA Z20 bias',
                                    'long_name': 'JJA SST bias relative to ORA-S4 1982-2017',
                                    'units': 'm',
                                    'model': 'CMIP5',
                                    'Scenario': 'RCP85',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})

bias_cmip5  = xr.Dataset({'bias': (['product'],z20_bias),
                       }
                       ,coords={'product':model},attrs={'standard_name': 'Z20 bias',
                                    'long_name': 'SST bias relative to HadISST 1982-2017',
                                    'units': 'm',
                                    'model': 'CMIP5',
                                    'Scenario': 'RCP85',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})


In [18]:

bias_jja_cmip5.to_netcdf(path_data+'z20_bias_atl3_CMIP5_jja_dtd_19822017.nc')
bias_cmip5.to_netcdf(path_data+'z20_bias_atl3_CMIP5_dtd_19822017.nc')

